In [2]:
import pandas as pd
import numpy as np

data = pd.read_csv('couples.csv', sep = ',',na_values = '.', decimal = '.', encoding =  'utf-8')

# Analyse préliminaires

- graph des variable en fct de y ( boite a moustache)
- tableau des freq
- 


In [12]:
data.dtypes

id                            int64
enfant                        int64
dconsultation        datetime64[ns]
dconception          datetime64[ns]
ddn                        category
age_h                         int64
diplome_h                  category
bmi_h                       float64
patho_h                    category
cryptorchidie              category
spermo                     category
age_f                       float64
diplome_f                  category
bh_f                       category
ct_f                       category
patho_f                    category
fecondite                  category
duree_infertilite           float64
traitement                 category
dtype: object

In [13]:
print("nous avons "+ str(len(data)) + " obs")

nous avons 1131 obs


Nous avons 19 variables, 6 d'entres elles sont de type numerique. 3 d'entre eux sont de type "date" et les 10 restantes sont de type catégorique(catégorielle ?), et cela, pour un total de 1131 observations.

In [32]:
# Transform

data["dconsultation"] = pd.to_datetime(data.dconsultation)
data["dconception"] = pd.to_datetime(data.dconsultation)
data["ddn"] = pd.to_datetime(data.ddn)

data = pd.concat([
        data.select_dtypes([], ['object']),
        data.select_dtypes(['object']).apply(pd.Series.astype, dtype='category')
        ], axis=1).reindex_axis(data.columns, axis=1)
data["enfant"] = data["enfant"].astype('category')

In [33]:
data.describe()

,id,age_h,bmi_h,age_f,duree_infertilite,isPathoH,isPathoF,delta_consult_concep
count,1131.000000,1131.000000,1109.000000,1128.000000,1131.000000,1131.000000,1069.000000,1131.0
mean,121258.500442,33.692308,24.821211,30.949468,31.540937,0.255526,0.212348,0.0
std,13060.822076,5.867785,3.484148,4.451067,22.587268,0.436349,0.409161,0.0
min,2707.000000,18.000000,11.830719,16.000000,1.000000,0.000000,0.000000,0.0
25%,115046.500000,30.000000,22.634676,28.000000,18.000000,0.000000,0.000000,0.0
50%,117233.000000,33.000000,24.386526,31.000000,24.000000,0.000000,0.000000,0.0
75%,119685.000000,37.000000,26.277285,34.000000,36.000000,1.000000,0.000000,0.0
max,142814.000000,61.000000,54.143211,47.000000,168.000000,1.000000,1.000000,0.0


La moyenne d'age pour les hommes est de 34 ans, avec un maximum de 61 ans et un minimum de 18 ans alors pour les femmes la moyenne tourne autour 32 ans avec un minimum de 16 ans et un maximum de 47 ans.

L'indice corporelle des hommes ayant consultés est en moyenne de 24.8, ils ont donc en moyenne une corpulence normale à la limite du surpoids. Cependant on retrouve des cas d'anorexie avec un IMC de 11.8 ou d'obésité massive avec un IMC de plus de 54.

En ce qui concerne la durée d'infertilite, celle-ci est en moyenne de 32 mois et peut aller jusqu'à 168 mois.


In [34]:
data.select_dtypes(['category']).describe()

,enfant,diplome_h,patho_h,cryptorchidie,spermo,diplome_f,bh_f,ct_f,patho_f,fecondite,traitement,tranche_age_h,tranche_age_f,tranche_bmi_h,tranche_duree_infertilite
count,1131,1128,1131,1131,1131,1109,772,776,763,1131,1131,1129,1127,1108,1130
unique,2,3,6,2,3,3,2,3,6,2,6,4,3,4,4
top,1,Bac+,non,Non,anormal,Bac+,normal,ovulation,non,primaire,ICSI,30 à 40 ans,30 à 40 ans,Corpulence normale,+ 2ans
freq,637,492,842,973,685,570,670,614,647,720,384,625,559,661,490


La variable "ddn", qui represente la date du dernier entretien téléphonique auquel le couple a participé, qui est unique (le 01/06/2009) nous indique qu'au moins 494 couples sont sans grossesse à cette date.
Les couples qui consultent, ont pour la plupart un niveau d'etudes supérieur que le bac et consultent pour un type d'infécondité primaire. 

Le type de traitement de l'infécondité reçu par le couple est souvent l'ICSI. En effet, chez l'homme, sur 1131 observations, 685 ont un sperme anormal. De plus, la plupart d'entre eux n'ont aucune pathologie et aucun problème de cryptorchidie.

Chez les femmes en revanche, la plupart ne presente aucune pathologie ainsi qu'un bilan hormonal normal et l'ovulation a lieu de façon normale et régulière.

Globalement, 637 couples sur nos 1131 observations ont réussit à concevoir un enfant.

Note : chez l'hommeil y  a 12 pathologie, à recoder.

# Recodage des données

### Age

In [24]:
data['tranche_age_h'] = pd.cut(data.age_h, bins = [data.age_h.min(),30,40,50,data.age_h.max()], labels = ['- 30 ans','30 à 40 ans', \
                                                                                       '40 à 50 ans','+ 50 ans'])
data['tranche_age_f'] = pd.cut(data.age_f, bins = [16,30,40,data.age_f.max()], labels = ['- 30 ans','30 à 40 ans', \
                                                                                       '+ 40 ans'])

### Indice de masse corporelle

In [22]:
data['tranche_bmi_h'] = pd.cut(data.bmi_h, \
                              bins = [data.bmi_h.min(),16.5,18.5,25,30,data.bmi_h.max()],\
                              labels = ['Denutrition ou anorexie','Maigreur', 'Corpulence normale',\
                                       'Surpoid', 'Obesite'])

### Encadrement de la durée d'infertilité avant la première consultation

In [25]:
data['tranche_duree_infertilite'] = pd.cut(data.duree_infertilite, \
                              bins = [data.duree_infertilite.min(),6,12,24,data.duree_infertilite.max()],\
                              labels = ['- 6 mois','6-12 mois', '1-2 ans','+ 2ans'])

### Pathologies

In [26]:
dico = {'non': 'non',
 'chimiotherapie': 'chimiotherapie',
 'neurologique' : 'probleme neurologique',
 'hodgkin , chimiotherapie , radiotherapie': 'chimiotherapie',
 'sinusites chroniques' : 'probleme respiratoire',
 'sarcome , chimiotherapie': 'chimiotherapie',
 'cancer testis , chimiotherapie' : 'chimiotherapie',
 'pathologies respiratoire chroniques': 'probleme respiratoire',
 'sinusites chroniques , pathologies respiratoire chroniques': 'probleme respiratoire',
 'chimiotherapie , radiotherapie' : 'chimiotherapie',
 'autre' : 'autre',
 'diabete' : 'diabete'}
data.patho_h = data.patho_h.map(dico)

Nous avons recodé les variables de façon à ...(expliquer)

# Ajout de features

 plutot prédire si les individus ont eu des enfants après la ddn (dans le cas ou la première consultation à eu lieu peu avant), il nous faut connaitre les durée entre traitement et consultation.


In [35]:
data['delta_consult_concep'] = (data.dconception - data.dconsultation)/np.timedelta64(1, 'M')
data['delta_consult_ddn'] = (data.ddn - data.dconsultation)/np.timedelta64(1, 'M')
data['duree_totale_du_programme'] = [i if not np.isnan(i) else j for i,j in zip(data['delta_consult_concep'],data['delta_consult_ddn'])]

In [30]:
dicoIsPathoH = {elem:0 if elem == 'non' else 1 for elem in list(set(data.patho_h))}

catPathoF = list(set(data.patho_f))
catPathoF = [x for x in catPathoF if str(x) != 'nan']


dicoIsPathoF = {elem:0 if elem == 'non' else 1 for elem in catPathoF}

data['isPathoH'] = data.patho_h.map(dicoIsPathoH)
data['isPathoF'] = data.patho_h.map(dicoIsPathoF)